In [270]:
!pip install eli5

In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import  cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [272]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix'

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [273]:
ls data

men_shoes.csv  shoes_prices.csv


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[ feats ].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [0]:
df['brand_cat'] = df['brand'].factorize()[0]

In [277]:
run_model(['brand_cat'])

(-58.38655694633361, 4.223555478221712)

In [278]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.47223572384038, 4.328288468270897)

##feature engineering

In [279]:
df.head()

,id,asins,brand,categories,colors,count,dateadded,dateupdated,descriptions,dimension,ean,features,flavors,imageurls,isbn,keys,manufacturer,manufacturernumber,merchants,name,prices_amountmin,prices_amountmax,prices_availability,prices_color,prices_condition,prices_count,prices_currency,prices_dateadded,prices_dateseen,prices_flavor,prices_issale,prices_merchant,prices_offer,prices_returnpolicy,prices_shipping,prices_size,prices_source,prices_sourceurls,prices_warranty,quantities,reviews,sizes,skus,sourceurls,upc,vin,websiteids,weight,brand_cat
0,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",39.89,39.89,NaN,NaN,NaN,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,True,NaN,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
1,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",51.99,51.99,NaN,NaN,new,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,False,UnbeatableSale - Walmart.com,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
2,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,40.02,40.02,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2016-03-08T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
3,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,50.31,50.31,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2015-11-30T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
4,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""

In [0]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]

In [281]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [282]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [283]:
df.features[0]

'[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'

In [284]:
literal_eval(df.features[0])

[{'key': 'Gender', 'value': ['Men']},
 {'key': 'Shoe Size', 'value': ['M']},
 {'key': 'Shoe Category', 'value': ["Men's Shoes"]},
 {'key': 'Color', 'value': ['Multicolor']},
 {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']},
 {'key': 'Brand', 'value': ['Josmo']}]

In [0]:
def parse_features(x):
  if str(x) == 'nan': return []
  return literal_eval(x.replace('\\"', '"'))

df['features_parsed'] = df['features'].map(parse_features)

In [286]:
df['features_parsed'].head().values

array([list([{'key': 'Gender', 'value': ['Men']}, {'key': 'Shoe Size', 'value': ['M']}, {'key': 'Shoe Category', 'value': ["Men's Shoes"]}, {'key': 'Color', 'value': ['Multicolor']}, {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']}, {'key': 'Brand', 'value': ['Josmo']}]),
       list([{'key': 'Gender', 'value': ['Men']}, {'key': 'Shoe Size', 'value': ['M']}, {'key': 'Shoe Category', 'value': ["Men's Shoes"]}, {'key': 'Color', 'value': ['Multicolor']}, {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']}, {'key': 'Brand', 'value': ['Josmo']}]),
       list([{'key': 'Gender', 'value': ['Men']}, {'key': 'Color', 'value': ['Black']}, {'key': 'Shipping Weight (in pounds)', 'value': ['0.45']}, {'key': 'Condition', 'value': ['New']}, {'key': 'Brand', 'value': ['SERVUS BY HONEYWELL']}, {'key': 'manufacturer_part_number', 'value': ['ZSR101BLMLG']}]),
       list([{'key': 'Gender', 'value': ['Men']}, {'key': 'Color', 'value': ['Black']}, {'key': 'Shipping Weight (in

In [0]:
# [
#  {'key': 'Gender', 'value': ['Men']}, 
#  {'key': 'Shoe Size', 'value': ['M']}, 
#  {'key': 'Shoe Category', 'value': ["Men's Shoes"]}, 
#  {'key': 'Color', 'value': ['Multicolor']}, 
#  {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']}, 
#  {'key': 'Brand', 'value': ['Josmo']}]

#  # chcemy mieć w postaci:

#  {
#     'Gender':'Men',
#     'Shoe Size':'M',
#     'Shoe Category':"Men's Shoes"
#  }


In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict
  
  features = literal_eval(x.replace('\\"', '"'))

  # item = {'key': 'Gender', 'value': ['Men']}

  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()
  
    output_dict[key] = value

  return output_dict


df['features_parsed'] = df['features'].map(parse_features)

In [288]:
df['features_parsed'].head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [289]:
keys = set()
df['features_parsed'].map( lambda x: keys.update(x.keys()) )
len(keys)

476

In [290]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map( lambda feats: feats[key] if key in feats else np.nan )

In [291]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_overall material', 'feat_item weight unit of measure',
       'feat_charger included', 'feat_hat style', 'feat_hooded',
       'feat_light transmission (vlt)', 'feat_seller inventory number',
       'feat_protects against', 'feat_temple size:', 'feat_battery type'],
      dtype='object', length=526)

In [292]:
df[ df['feat_athlete'].isnull() ].shape

(18272, 526)

In [293]:
df.shape[0]

18280

In [294]:
df[ False == df['feat_athlete'].isnull() ].shape[0] / df.shape[0] * 100

0.0437636761487965

In [0]:
keys_stat = {}

for key in keys:
  keys_stat[key] = df[ False == df[get_name_feat(key)].isnull() ].shape[0] / df.shape[0] * 100

In [296]:
{k:v for k,v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]


# Spróbujemy wszystkie

for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]
  

In [298]:
df[ df.brand != df.feat_brand ].shape

(18228, 1002)

In [299]:
df[ df.brand == df.feat_brand ].shape

(52, 1002)

In [300]:
df[ df.brand == df.feat_brand ][['brand', 'feat_brand']].head()

,brand,feat_brand
2163,adidas,adidas
2417,adidas,adidas
3021,adidas,adidas
3054,adidas,adidas
3629,totes,totes


In [301]:
df[ df.brand != df.feat_brand ][['brand', 'feat_brand']].head()

,brand,feat_brand
0,Josmo,josmo
1,Josmo,josmo
2,SERVUS BY HONEYWELL,servus by honeywell
3,SERVUS BY HONEYWELL,servus by honeywell
4,SERVUS BY HONEYWELL,servus by honeywell


In [0]:
df['brand'] = df['brand'].map(lambda x: str(x).lower())

In [303]:
df[ df.brand == df.feat_brand ].shape

(8846, 1002)

In [304]:
df[ df.brand != df.feat_brand ][['brand', 'feat_brand']].head()

,brand,feat_brand
21,rubies,generic
22,rubies,generic
23,rubies,generic
24,unbranded,NaN
31,american fighter,NaN


In [305]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(['brand_cat'], model)

(-57.3766492589775, 4.139470602725344)

In [306]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(['brand_cat', 'feat_brand_cat'], model)

(-56.91526138531644, 4.499982156156148)

In [307]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(['brand_cat', 'feat_brand_cat', 'feat_color_cat'], model)

(-57.50331016317796, 5.092137903407483)

In [308]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(['brand_cat', 'feat_brand_cat', 'feat_color_cat', 'feat_gender_cat'], model)

(-57.87252082679312, 4.698883273486404)

In [309]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(['brand_cat', 'feat_brand_cat', 'feat_color_cat', 'feat_gender_cat', 
           'feat_manufacturer part number_cat'], model)

(-57.803265896279264, 4.757094660356917)

In [310]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_color_cat', 'feat_gender_cat', 
           'feat_manufacturer part number_cat', 'feat_material_cat']

model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats, model)

(-57.05469060980265, 4.275264188498871)

In [0]:
X = df[feats].values
y = df['prices_amountmin'].values

In [312]:
m = RandomForestRegressor(max_depth= 5, n_estimators=100, random_state=0)

m.fit(X, y)

perm = PermutationImportance(m, random_state=1).fit(X,y)
eli5.show_weights(perm, feature_names = feats)

Weight,Feature
0.2565 ± 0.0110,brand_cat
0.1057 ± 0.0095,feat_material_cat
0.0431 ± 0.0043,feat_gender_cat
0.0204 ± 0.0010,feat_brand_cat
0.0072 ± 0.0004,feat_color_cat
0.0008 ± 0.0001,feat_manufacturer part number_cat


In [313]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_color_cat', 'feat_gender_cat', 'feat_material_cat']
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats, model)

(-57.10871101417372, 4.164974940751901)

In [314]:
X = df[feats].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth= 5, n_estimators=100, random_state=0)

m.fit(X, y)

perm = PermutationImportance(m, random_state=1).fit(X,y)
eli5.show_weights(perm, feature_names = feats)

Weight,Feature
0.2581 ± 0.0103,brand_cat
0.1015 ± 0.0115,feat_material_cat
0.0438 ± 0.0020,feat_gender_cat
0.0204 ± 0.0003,feat_brand_cat
0.0065 ± 0.0013,feat_color_cat


In [315]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat']
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats, model)

(-57.10809963091357, 4.166407565465867)

In [316]:
df['brand'].value_counts(normalize = True)

nike            0.097210
puma            0.033315
ralph lauren    0.028775
vans            0.021116
new balance     0.020295
                  ...   
care apparel    0.000055
nikon           0.000055
xray, xray      0.000055
kingshow        0.000055
gitzo           0.000055
Name: brand, Length: 1732, dtype: float64

In [317]:
df[ df['brand'] == 'nike' ].features_parsed.head().values

array([{'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'style': 'athletic sneakers', 'condition': 'new with box'}, {}],
      dtype=object)

In [318]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat','feat_style_cat']
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats, model)

(-57.11346973110129, 4.305165982836422)

In [319]:
X = df[feats].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth= 5, n_estimators=100, random_state=0)

m.fit(X, y)

perm = PermutationImportance(m, random_state=1).fit(X,y)
eli5.show_weights(perm, feature_names = feats)

Weight,Feature
0.2605 ± 0.0099,brand_cat
0.1070 ± 0.0093,feat_material_cat
0.0452 ± 0.0026,feat_gender_cat
0.0207 ± 0.0011,feat_brand_cat
0.0064 ± 0.0007,feat_style_cat


In [320]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat',
         'feat_style_cat', 'feat_sport_cat']
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats, model)

(-57.14725329454147, 4.233263237954002)

In [321]:
X = df[feats].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth= 5, n_estimators=100, random_state=0)

m.fit(X, y)

perm = PermutationImportance(m, random_state=1).fit(X,y)
eli5.show_weights(perm, feature_names = feats)

Weight,Feature
0.2600 ± 0.0110,brand_cat
0.1047 ± 0.0084,feat_material_cat
0.0458 ± 0.0047,feat_gender_cat
0.0213 ± 0.0011,feat_brand_cat
0.0069 ± 0.0016,feat_style_cat
0.0002 ± 0.0000,feat_sport_cat


In [322]:
df[ df['brand'] == 'nike' ].features_parsed.sample(5).values

array([{'sport': 'football', 'main color': 'purple & green', 'type': 'cleats'},
       {'sport': 'soccer', 'condition': 'new with box'},
       {'material': 'mesh', 'gender': 'men', 'size': '9', 'shoe category': 'basketball', 'fabric content': '100% mesh', 'color': 'black/gym red-white', 'model': '845098-001', 'casual & dress shoe style': 'comfort', 'features': 'zoom air units midsole ,�� rubber outsole with traction ,�� one-piece woven textile upper', 'brand': 'nike', 'manufacturer_part_number': '845098-001', 'shoe width': 'd (standard width)'},
       {'sport': 'running shoes', 'occasion': 'running shoes', 'material': 'suede', 'gender': 'men', 'shoe category': "men's shoes", 'assembled product dimensions (l x w x h)': '12.00 x 8.00 x 6.00 inches', 'fabric content': 'suede', 'color': 'sport royal/vrsty mz/white/blk', 'model': '642402 407', 'shoe closure': 'lace-up', 'casual & dress shoe style': 'running shoes', 'manufacturer part number': '642402 407', 'variant group id': '224#mp#6424

In [0]:
feats_cat = [x for x in df.columns if 'cat' in x]

In [324]:
feats_cat

['categories',
 'brand_cat',
 'feat_recommended location',
 'feat_multi pack indicator',
 'feat_shoe category',
 'feat_certifications and listings',
 'feat_catalog',
 'feat_clothing category',
 'feat_location - city/state',
 'feat_location - country',
 'feat_fabrication',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_manufacturer part number_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_target audience_cat',
 'feat_band width_cat',
 'feat_style #_cat',
 'feat_manufacturer_cat',
 'feat_part type_cat',
 'feat_fastener type_cat',
 'feat_construction_cat',
 'feat_auto_cat',
 'feat_dial markers_cat',
 'feat_case diameter_cat',
 'feat_issued/ not-issued_cat',
 'feat_model number_cat',
 'feat_size/dimensions_cat',
 'feat_number of heat settings_cat',
 'feat_authenticity_cat',
 'feat_date first available at amazon.ca_cat',
 'feat_recommended location_cat',
 'feat_high visibility (ansi compliant)_cat',
 'feat_fit:_cat',
 'feat_number of items_cat',


In [325]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat','feat_style_cat',
         'feat_shape_cat', 'feat_metal type_cat', 'feat_occasion_cat', 'feat_fabric content_cat',
         'feat_jacket length_cat', 'feat_size_cat',  'feat_color_cat']


model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats, model)

(-57.291457759081766, 4.197112192419773)

In [326]:
X = df[feats].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth= 5, n_estimators=100, random_state=0)

m.fit(X, y)

perm = PermutationImportance(m, random_state=1).fit(X,y)
eli5.show_weights(perm, feature_names = feats)

Weight,Feature
0.2495 ± 0.0100,brand_cat
0.0951 ± 0.0051,feat_material_cat
0.0260 ± 0.0034,feat_gender_cat
0.0234 ± 0.0016,feat_brand_cat
0.0144 ± 0.0008,feat_fabric content_cat
0.0121 ± 0.0008,feat_shape_cat
0.0079 ± 0.0011,feat_metal type_cat
0.0078 ± 0.0006,feat_color_cat
0.0038 ± 0.0003,feat_jacket length_cat
0.0029 ± 0.0006,feat_occasion_cat


In [327]:
df['weight'].unique()

array([nan, '3.0 lbs', '9 g', '1.45 lbs', '0.45 lbs', '1.0 lbs',
       '0.23 lbs', '5.0 lbs', '5.5 lbs', '7.45 lbs', '4.0 lbs',
       '2.7969 lbs', '3.9 lbs', '4.6 pounds', '2.1 lbs', '1.1057 lbs',
       '15.0 lbs', '2.4 ounces', '454 g', '0.105 lbs', '9.1 ounces',
       '4.8 lbs', '6.1 lbs', '6.5 lbs', '1.1041 lbs', '1.3 Kg', '91 g',
       '20.0 lbs', '6.0 lbs', '386 g', '0.81 lbs', '4.5 lbs',
       '0.5 ounces', '2.0 lbs', '3.13 lbs', '5.9 lbs', '6.15 lbs',
       '1 pounds', '1.95 lbs', '2.15 lbs', '2 pounds', '2.1 pounds',
       '14 Kg', '0.4788 lbs', '10.0 lbs', '0.38 lbs', '2.5 lbs',
       '68.912 lbs', '45 g', '13.09 lbs', '2.5 pounds', '0.21 lbs',
       '16.75 lbs', '6.3 lbs', '272 g', '1.8 Kg', '2.8 pounds', '0.1 lbs',
       '5.05 lbs', '0.28 lbs', '76.08 lbs', '0.15 lbs', '200 g',
       '7.8 pounds', '399 g', '4.95 lbs', '64.144 lbs', '24 pounds',
       '73.696 lbs', '1.6 lbs', '6.6 ounces', '5 g', '1.2 Kg', '862 g',
       '3.05 lb', '8.6 ounces', '3.6 lbs', '71.

In [0]:
# lbs (pounds) na gramy  * 453.59237
# ounces na gramy * 28.3495231

df['weight'] = df['weight'].map(lambda x: str(x).lower())

In [329]:
df['weight'].unique()

array(['nan', '3.0 lbs', '9 g', '1.45 lbs', '0.45 lbs', '1.0 lbs',
       '0.23 lbs', '5.0 lbs', '5.5 lbs', '7.45 lbs', '4.0 lbs',
       '2.7969 lbs', '3.9 lbs', '4.6 pounds', '2.1 lbs', '1.1057 lbs',
       '15.0 lbs', '2.4 ounces', '454 g', '0.105 lbs', '9.1 ounces',
       '4.8 lbs', '6.1 lbs', '6.5 lbs', '1.1041 lbs', '1.3 kg', '91 g',
       '20.0 lbs', '6.0 lbs', '386 g', '0.81 lbs', '4.5 lbs',
       '0.5 ounces', '2.0 lbs', '3.13 lbs', '5.9 lbs', '6.15 lbs',
       '1 pounds', '1.95 lbs', '2.15 lbs', '2 pounds', '2.1 pounds',
       '14 kg', '0.4788 lbs', '10.0 lbs', '0.38 lbs', '2.5 lbs',
       '68.912 lbs', '45 g', '13.09 lbs', '2.5 pounds', '0.21 lbs',
       '16.75 lbs', '6.3 lbs', '272 g', '1.8 kg', '2.8 pounds', '0.1 lbs',
       '5.05 lbs', '0.28 lbs', '76.08 lbs', '0.15 lbs', '200 g',
       '7.8 pounds', '399 g', '4.95 lbs', '64.144 lbs', '24 pounds',
       '73.696 lbs', '1.6 lbs', '6.6 ounces', '5 g', '1.2 kg', '862 g',
       '3.05 lb', '8.6 ounces', '3.6 lbs', '7

In [0]:
def wagi(x):
  if ' lbs' in x or ' pounds' in x or 'lb' in x:
    return float(x.split()[0]) * 453.59237
  if ' ounces' in x:
    return float(x.split()[0]) * 28.3495231
  if ' kg' in x:
    return float(x.split()[0]) * 1000
  if ' g' in x:
    return float(x.split()[0])
  else:
    return x

In [331]:
float(df['weight'][32].split()[0]) * 453.59237

1360.77711

In [0]:
df['weight_norm'] = df['weight'].map(wagi)

In [333]:
df.weight_norm.unique()

array(['nan', 1360.77711, 9.0, 657.7089365, 204.1165665, 453.59237,
       104.32624510000001, 2267.96185, 2494.7580350000003, 3379.2631565,
       1814.36948, 1268.652499653, 1769.010243, 2086.524902, 952.543977,
       501.53708350899996, 6803.88555, 68.03885543999999, 454.0,
       47.62719885, 257.98066021, 2177.243376, 2766.913457, 2948.350405,
       500.8113357170001, 1300.0, 91.0, 9071.8474, 2721.55422, 386.0,
       367.4098197, 2041.165665, 14.17476155, 907.18474, 1419.7441181,
       2676.1949830000003, 2789.5930755000004, 884.5051215, 975.2235955,
       14000.0, 217.18002675600002, 4535.9237, 172.3651006, 1133.980925,
       31257.957401440002, 45.0, 5937.5241233, 95.2543977, 7597.6721975,
       2857.631931, 272.0, 1800.0, 1270.058636, 45.35923700000001,
       2290.6414685, 127.00586360000001, 34509.3075096, 68.0388555, 200.0,
       3538.020486, 399.0, 2245.2822315000003, 29095.228981280005,
       10886.21688, 33427.94329952, 725.7477920000001, 187.10685245999997,
    

In [334]:
df.weight_norm.value_counts()

nan                   17710
453.59237               121
1360.77711               92
1270.058636              70
2267.96185               48
                      ...  
386.0                     1
975.2235955               1
2494.7580350000003        1
2789.5930755000004        1
5.0                       1
Name: weight_norm, Length: 78, dtype: int64

In [0]:
df['weight_norm'] = df.weight_norm.astype(np.float)

In [336]:
df.weight_norm.mean()

2019.1198054150984

In [0]:
df['weight_norm'].fillna(df.weight_norm.mean(), inplace=True)

In [338]:
df.weight_norm.mean()

2019.1198054147828

In [0]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat','feat_style_cat',
         'feat_shape_cat', 'feat_metal type_cat', 'feat_fabric content_cat',
         'feat_size_cat',  'feat_color_cat']


model = RandomForestRegressor(max_depth=5, n_estimators=200)
result = run_model(feats, model)

In [340]:
X = df[feats].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth= 5, n_estimators=200, random_state=0)

m.fit(X, y)

print(result)
perm = PermutationImportance(m, random_state=1).fit(X,y)
eli5.show_weights(perm, feature_names = feats)

(-57.2376930709411, 4.09330355154395)


Weight,Feature
0.2613 ± 0.0072,brand_cat
0.1007 ± 0.0064,feat_material_cat
0.0245 ± 0.0030,feat_gender_cat
0.0222 ± 0.0024,feat_brand_cat
0.0145 ± 0.0009,feat_fabric content_cat
0.0118 ± 0.0009,feat_shape_cat
0.0081 ± 0.0011,feat_metal type_cat
0.0079 ± 0.0016,feat_color_cat
0.0033 ± 0.0007,feat_style_cat
0.0005 ± 0.0002,feat_size_cat


In [0]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat','feat_style_cat',
         'feat_shape_cat', 'feat_metal type_cat', 'feat_occasion_cat', 'feat_fabric content_cat',
         'feat_size_cat',  'feat_color_cat', 'weight_norm', 'feat_sport_cat']


model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats, model)

In [342]:
X = df[feats].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth= 5, n_estimators=100, random_state=0)

m.fit(X, y)

print(result)
perm = PermutationImportance(m, random_state=1).fit(X,y)
eli5.show_weights(perm, feature_names = feats)

(-57.21489232426124, 4.219647466658649)


Weight,Feature
0.2546 ± 0.0095,brand_cat
0.0981 ± 0.0054,feat_material_cat
0.0249 ± 0.0017,feat_gender_cat
0.0227 ± 0.0040,feat_brand_cat
0.0138 ± 0.0011,feat_fabric content_cat
0.0117 ± 0.0007,feat_shape_cat
0.0087 ± 0.0017,feat_metal type_cat
0.0072 ± 0.0005,feat_color_cat
0.0031 ± 0.0006,feat_occasion_cat
0.0027 ± 0.0009,feat_style_cat


In [0]:
ls matrix_one/

In [0]:
!git add matrix_one/day5.ipynb

In [0]:
!git config --global user.email